# MNIST Tutorial
Here be the MNIST Tutorial! For now we'll just load up the data and maybe look at it.

For now, we're looking at this [link.](https://www.digitalocean.com/community/tutorials/mnist-dataset-in-python)

In [4]:
from tensorflow.keras.datasets import mnist

# LOAD THE DATA?!
(train_X, train_y), (test_X, test_y) = mnist.load_data()

print('X_train: ' + str(train_X.shape))
print('Y_train: ' + str(train_y.shape))
print('X_test:  '  + str(test_X.shape))
print('Y_test:  '  + str(test_y.shape))

11490434/11490434 [==============================] - 3s 0us/step
X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


And now, to actually do tutorial related stuff with it: